In [2]:
import torch

from torch.utils.data import DataLoader
from torchvision import datasets, transforms


def load_data(split, batch_size, num_workers):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

#     data = datasets.ImageNet(root="~/data/datasets/imagenet", split=split, transform=preprocess)

    data = datasets.CIFAR10(root="~/data/datasets/CIFAR10", train=True if split == "train" else False, transform=preprocess)
    loader = DataLoader(data, batch_size=batch_size, num_workers=num_workers)

    return loader

In [31]:
def predict(model, data_loader, device):
    model.eval()

    y_true = torch.empty(0, device=device)
    y_pred = torch.empty(0, device=device)

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)

            output = model(images)
            
            probabilities = torch.nn.functional.softmax(output, dim=1) # type: ignore
            _, pred = torch.topk(probabilities, 5)

            y_true = torch.cat((y_true, labels), 0)
            y_pred = torch.cat((y_pred, pred), 0)

            print(f'run -- {labels} {output} {probabilities} {pred}')
    return y_true, y_pred

In [32]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
data_loader = load_data("train", 2, 1)
predict(model, data_loader, 'cpu')

Using cache found in /home/kaa716/.cache/torch/hub/pytorch_vision_v0.10.0


run -- tensor([6, 9]) tensor([[ 1.5302,  2.9089, -3.6547,  ...,  2.7869,  1.6106, -3.6487],
        [ 0.2348,  0.2972, -1.1674,  ..., -3.9148, -2.5432, -4.1073]]) tensor([[2.6692e-04, 1.0596e-03, 1.4950e-06,  ..., 9.3792e-04, 2.8928e-04,
         1.5039e-06],
        [3.4161e-06, 3.6364e-06, 8.4060e-07,  ..., 5.3876e-08, 2.1236e-07,
         4.4441e-08]]) tensor([[ 43, 335, 220, 356,  62],
        [675, 867, 660, 705, 408]])
run -- tensor([9, 4]) tensor([[ 3.0495, -1.2110,  0.1561,  ..., -1.8464, -0.2908, -6.2380],
        [ 2.8547, -1.2955, -1.9502,  ...,  0.0096, -2.0801, -3.5783]]) tensor([[6.2163e-04, 8.7747e-06, 3.4430e-05,  ..., 4.6480e-06, 2.2022e-05,
         5.7550e-08],
        [4.2077e-04, 6.6319e-06, 3.4460e-06,  ..., 2.4460e-05, 3.0263e-06,
         6.7644e-07]]) tensor([[856, 510, 491, 821, 339],
        [101, 291, 385, 278, 275]])
run -- tensor([1, 1]) tensor([[ 2.7154, -2.5675, -0.5192,  ..., -4.6144, -1.9632, -3.0122],
        [-1.6114,  4.4991, -1.6218,  ..., -0.2035,

KeyboardInterrupt: 

In [64]:
for images, labels in data_loader:
    output = model(images)

KeyboardInterrupt: 

In [65]:
output
# mask = torch.ones_like(output)
# mask[:,10:] = 0
# output = output * mask
# output

tensor([[-1.9578, -0.5072, -3.0716,  ..., -2.2870, -0.6515, -0.9140],
        [ 3.8113,  0.3694, -2.2185,  ..., -0.3533,  0.6191, -4.3903]],
       grad_fn=<AddmmBackward>)

In [3]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model

Using cache found in /home/kaa716/.cache/torch/hub/pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [21]:
model.classifier[6]
model.state_dict()['classifier.6.weight']

tensor([[ 0.0327, -0.0062, -0.0040,  ...,  0.0160,  0.0456, -0.0158],
        [-0.0281,  0.0393, -0.0035,  ..., -0.0250,  0.0265, -0.0159],
        [-0.0019, -0.0004, -0.0081,  ..., -0.0093,  0.0203, -0.0136],
        ...,
        [-0.0249, -0.0350,  0.0131,  ..., -0.0082,  0.0454, -0.0043],
        [ 0.0252, -0.0026, -0.0109,  ..., -0.0091, -0.0615, -0.0009],
        [-0.0039,  0.0090, -0.0018,  ...,  0.0229,  0.0042,  0.0185]])

In [11]:
model2 = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model2.classifier[6] = torch.nn.Linear(4096, 10)
model2

Using cache found in /home/kaa716/.cache/torch/hub/pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [31]:
model.state_dict()['classifier.6.weight']

tensor([[ 0.0327, -0.0062, -0.0040,  ...,  0.0160,  0.0456, -0.0158],
        [-0.0281,  0.0393, -0.0035,  ..., -0.0250,  0.0265, -0.0159],
        [-0.0019, -0.0004, -0.0081,  ..., -0.0093,  0.0203, -0.0136],
        ...,
        [-0.0249, -0.0350,  0.0131,  ..., -0.0082,  0.0454, -0.0043],
        [ 0.0252, -0.0026, -0.0109,  ..., -0.0091, -0.0615, -0.0009],
        [-0.0039,  0.0090, -0.0018,  ...,  0.0229,  0.0042,  0.0185]])

In [36]:
a = torch.tensor([0.0, 0.0, 0.0, 1.0])
sm = torch.nn.functional.softmax(a)

/home/kaa716/.conda/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [38]:
torch.topk(sm, 1)

torch.return_types.topk(
values=tensor([0.4754]),
indices=tensor([3]))

In [39]:
sm

tensor([0.1749, 0.1749, 0.1749, 0.4754])

In [49]:
maxk = max((1,))
a.topk(maxk)

torch.return_types.topk(
values=tensor([1.]),
indices=tensor([3]))

In [51]:
torch.argmax(a)

tensor(3)